<a href="https://colab.research.google.com/github/MohamedFarhun/DeterminedAI-HotelBookingCancellation/blob/main/Determined_AI_Booking_Cancellation_Exploratory_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install sidetable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
!pip install determined

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
! git clone https://github.com/determined-ai/determined

fatal: destination path 'determined' already exists and is not an empty directory.


**Distributed training with determined AI:-** You may quickly distribute model training across your cluster to enable more effective experimentation and model building at an earlier stage. With significant performance improvements over competing solutions, Determined uses synchronous, data-parallel distributed training.

In [5]:
import boto3
import os
def download_data_from_s3(self):
    s3_bucket = self.context.get_data_config()["bucket"]
    download_directory = f"/tmp/data-rank{self.context.distributed.get_rank()}"
    data_file = "DeterminedHotelBookingcancellation_exploratory.csv"
    s3 = boto3.client("s3")
    os.makedirs(download_directory, exist_ok=True)
    filepath = os.path.join(download_directory, data_file)
    if not os.path.exists(filepath):
        s3.download_file(s3_bucket, data_file, filepath)
    return download_directory

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sidetable as stb
import warnings 
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', None)
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('DeterminedHotelBookingcancellation_exploratory.csv')
df.head()

,BookingID,LeadTime,ArrivalYear,ArrivalMonth,ArrivalDate,MealPlan,Parking,RoomType,MarketSegment,RepeatedGuest,NumPrevCancellations,NumPreviousNonCancelled,AvgRoomPrice,SpecialRequests,BookingStatus,guests,arrival_date,total_stays
0,1,10,2018,3,31,Meal Plan 1,0,Room_Type 1,Corporate,0,0,0,95.00,0,0,1,2018-3-31,1
1,2,116,2018,2,28,Meal Plan 1,0,Room_Type 1,Online,0,0,0,61.00,0,0,1,2018-2-28,3
2,3,11,2018,7,25,Meal Plan 1,0,Room_Type 1,Online,0,0,0,129.75,1,1,3,2018-7-25,3
3,4,3,2017,9,12,Meal Plan 1,0,Room_Type 1,Online,0,0,0,152.00,3,1,2,2017-9-12,1
4,5,28,2018,3,7,Meal Plan 1,0,Room_Type 1,Offline,0,0,0,87.00,0,1,2,2018-3-7,4


In [8]:
df.isna().sum()

BookingID                  0
LeadTime                   0
ArrivalYear                0
ArrivalMonth               0
ArrivalDate                0
MealPlan                   0
Parking                    0
RoomType                   0
MarketSegment              0
RepeatedGuest              0
NumPrevCancellations       0
NumPreviousNonCancelled    0
AvgRoomPrice               0
SpecialRequests            0
BookingStatus              0
guests                     0
arrival_date               0
total_stays                0
dtype: int64

In [9]:
df.describe(include = 'O')

,MealPlan,RoomType,MarketSegment,arrival_date
count,29020,29020,29020,29020
unique,4,7,5,550
top,Meal Plan 1,Room_Type 1,Online,2018-10-13
freq,22249,22470,18593,198


In [10]:
df.stb.freq(['BookingStatus'], cum_cols = False)
#0 - cancelled
#1 - not cancelled

,BookingStatus,count,percent
0,1,19500,67.195038
1,0,9520,32.804962


In [11]:
lead_time_months = df['LeadTime'] // 30
lead_time_months.value_counts() / len(df)
df['lead_time_month'] = df['LeadTime'] // 30
df.stb.freq(['lead_time_month'], cum_cols = False)

,lead_time_month,count,percent
0,0,9709,33.456237
1,1,5080,17.505169
2,2,3711,12.787733
3,3,2775,9.562371
4,4,1798,6.195727
5,5,1644,5.665059
6,6,1264,4.355617
7,8,752,2.591316
8,7,744,2.563749
9,9,646,2.226051


In [12]:
df.stb.freq(['MarketSegment'], cum_cols= False)

,MarketSegment,count,percent
0,Online,18593,64.069607
1,Offline,8412,28.986906
2,Corporate,1596,5.499655
3,Complementary,319,1.099242
4,Aviation,100,0.344590


In [13]:
df.stb.freq(['MealPlan'], cum_cols=False)

,MealPlan,count,percent
0,Meal Plan 1,22249,76.667815
1,Not Selected,4123,14.207443
2,Meal Plan 2,2643,9.107512
3,Meal Plan 3,5,0.017229


In [14]:
df.stb.freq(['RepeatedGuest'], cum_cols = False)

,RepeatedGuest,count,percent
0,0,28276,97.436251
1,1,744,2.563749


In [15]:
df.stb.freq(['NumPrevCancellations'], cum_cols = False)

,NumPrevCancellations,count,percent
0,0,28744,99.048932
1,1,155,0.534114
2,2,43,0.148174
3,3,33,0.113715
4,11,21,0.072364
5,5,11,0.037905
6,4,8,0.027567
7,13,4,0.013784
8,6,1,0.003446


In [16]:
def cancellation(row):
    if (row['NumPrevCancellations'] == 0):
        return 0
    else :
        return 1
df['is_previously_cancelled'] = df.apply(cancellation, axis = 1)
df.stb.freq(['is_previously_cancelled'], cum_cols = False)

,is_previously_cancelled,count,percent
0,0,28744,99.048932
1,1,276,0.951068


In [17]:
df.stb.freq(['SpecialRequests'], cum_cols = False)

,SpecialRequests,count,percent
0,0,15814,54.493453
1,1,9098,31.350793
2,2,3492,12.033081
3,3,550,1.895245
4,4,60,0.206754
5,5,6,0.020675


In [18]:
df.stb.freq(['arrival_date'], cum_cols = False)

,arrival_date,count,percent
0,2018-10-13,198,0.682288
1,2017-10-16,186,0.640937
2,2017-9-18,179,0.616816
3,2018-6-15,174,0.599586
4,2018-6-24,167,0.575465
...,...,...,...
545,2017-7-4,2,0.006892
546,2017-7-3,2,0.006892
547,2017-7-12,2,0.006892
548,2017-7-24,1,0.003446


In [19]:
from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))
     

In [20]:
pd.crosstab(df['guests'], df['BookingStatus'], normalize = 'index')

df.stb.freq(['guests'], cum_cols = False)

display_side_by_side([pd.crosstab(df['guests'], df['BookingStatus'], normalize = 'index'), df.stb.freq(['guests'], cum_cols = False)],['Cancelation', 'Number of Bookings'])


In [21]:
pd.crosstab(df['LeadTime'], df['BookingStatus'], normalize = 'index')

BookingStatus,0,1
LeadTime,,
0,0.052582,0.947418
1,0.103687,0.896313
2,0.061896,0.938104
3,0.095808,0.904192
4,0.102564,0.897436
...,...,...
381,1.000000,0.000000
386,0.941176,0.058824
418,1.000000,0.000000


In [22]:
pd.crosstab(df['arrival_date'], df['BookingStatus'], normalize = 'index').sort_values(1, ascending = False)

BookingStatus,0,1
arrival_date,,
2018-1-27,0.0,1.0
2017-8-24,0.0,1.0
2017-8-23,0.0,1.0
2017-12-17,0.0,1.0
2017-12-16,0.0,1.0
...,...,...
2017-7-14,1.0,0.0
2017-7-15,1.0,0.0
2017-7-26,1.0,0.0


In [23]:
pd.crosstab(df['guests'], df['BookingStatus'], normalize = 'index').sort_values(1, ascending = False)

BookingStatus,0,1
guests,,
10,0.000000,1.000000
12,0.000000,1.000000
1,0.241150,0.758850
5,0.307692,0.692308
2,0.345819,0.654181
3,0.360026,0.639974
4,0.443828,0.556172
11,1.000000,0.000000


In [26]:
pd.crosstab(df['MealPlan'], df['BookingStatus'], normalize = 'index')

BookingStatus,0,1
MealPlan,,
Meal Plan 1,0.313273,0.686727
Meal Plan 2,0.448354,0.551646
Meal Plan 3,0.200000,0.800000
Not Selected,0.330827,0.669173


In [27]:
pd.crosstab(df['MarketSegment'], df['BookingStatus'], normalize = 'index')

BookingStatus,0,1
MarketSegment,,
Aviation,0.340000,0.660000
Complementary,0.000000,1.000000
Corporate,0.106516,0.893484
Offline,0.300285,0.699715
Online,0.365191,0.634809


In [25]:
pd.crosstab(df['NumPrevCancellations'], df['BookingStatus'], normalize = 'index')

BookingStatus,0,1
NumPrevCancellations,,
0,0.330712,0.669288
1,0.058065,0.941935
2,0.000000,1.000000
3,0.030303,0.969697
4,0.000000,1.000000
5,0.000000,1.000000
6,0.000000,1.000000
11,0.000000,1.000000
13,1.000000,0.000000


In [28]:
pd.crosstab(df['SpecialRequests'], df['BookingStatus'], normalize = 'index')

BookingStatus,0,1
SpecialRequests,,
0,0.432402,0.567598
1,0.237964,0.762036
2,0.148053,0.851947
3,0.000000,1.000000
4,0.000000,1.000000
5,0.000000,1.000000


In [29]:
pd.crosstab(df['Parking'], df['BookingStatus'], normalize='index')

BookingStatus,0,1
Parking,,
0,0.334981,0.665019
1,0.109742,0.890258


In [30]:
def parking(row):
    if (row['Parking'] < 1):
        return 0
    else :
        return 1
    
df['parking_space'] = df.apply(parking, axis = 1)
df.stb.freq(['parking_space'], cum_cols = False)

,parking_space,count,percent
0,0,28127,96.922812
1,1,893,3.077188


In [31]:
pd.crosstab(df['guests'], df['BookingStatus'], values = df['LeadTime'], aggfunc = 'median')

BookingStatus,0,1
guests,,
1,96.0,15.0
2,146.0,44.0
3,105.0,50.0
4,68.5,29.0
5,114.0,15.0
10,NaN,11.0
11,8.0,NaN
12,NaN,37.0


In [32]:
pd.crosstab(df['NumPrevCancellations'], df['MarketSegment'], normalize = 'index')

MarketSegment,Aviation,Complementary,Corporate,Offline,Online
NumPrevCancellations,,,,,
0,0.003409,0.009880,0.050689,0.290704,0.645317
1,0.000000,0.051613,0.477419,0.329032,0.141935
2,0.046512,0.325581,0.604651,0.023256,0.000000
3,0.000000,0.212121,0.757576,0.000000,0.030303
4,0.000000,0.750000,0.250000,0.000000,0.000000
5,0.000000,0.000000,1.000000,0.000000,0.000000
6,0.000000,0.000000,1.000000,0.000000,0.000000
11,0.000000,0.000000,0.000000,0.000000,1.000000
13,0.000000,0.000000,0.000000,1.000000,0.000000


In [42]:
lead_time_canceled =  pd.crosstab(df['LeadTime'], df['BookingStatus'], normalize = 'index')
fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=lead_time_canceled.index, y=lead_time_canceled[0]),
    go.Bar(name='Cancelled Booking', x=lead_time_canceled.index, y=lead_time_canceled[1],marker_color = 'crimson')
])
fig.update_layout(barmode='group', title = 'Percentage of Confirmed Booking and Cancelled Booking per Lead Time Month',
                 yaxis=dict(
                 title='Percentage',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Lead Time Month',
                 titlefont_size=16,
                 tickfont_size=14,))
fig.show()

In [43]:
pd.crosstab(df['LeadTime'], df['BookingStatus']).sort_index(ascending = True)[1:10]

BookingStatus,0,1
LeadTime,,
1,90,778
2,32,485
3,48,453
4,52,455
5,54,397
6,50,369
7,33,301
8,44,309
9,35,219


In [44]:
guest_cancellation = pd.crosstab(df['guests'], df['BookingStatus'], normalize = 'index')
fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=guest_cancellation.index, y=guest_cancellation[0], marker_color = 'forestgreen'),
    go.Bar(name='Cancelled Booking', x=guest_cancellation.index, y=guest_cancellation[1],marker_color = 'crimson')])
fig.update_layout(barmode='group', title = 'Percentage of Cancellation Per Number of guest',
                 yaxis=dict(
                 title='Cancellation Rate (%) / Each Numbers of guest',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Number of guest',
                 titlefont_size=16,
                 tickfont_size=14,))

fig.show()

In [45]:
market_cancellation = pd.crosstab(df['MarketSegment'], df['BookingStatus'], normalize = 'index')
fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=market_cancellation.index, y=market_cancellation[0]),
    go.Bar(name='Cancelled Booking', x=market_cancellation.index, y=market_cancellation[1],marker_color = 'crimson')])
fig.update_layout(barmode='group', title = 'Percentage of Booking Per Market Segment',
                 yaxis=dict(
                 title='Percentage of booking (%) / Market Segment',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Market Segment',
                 titlefont_size=16,
                 tickfont_size=14,))

fig.show()

In [46]:
repeated_cancelled = pd.crosstab(df['RepeatedGuest'], df['BookingStatus'], normalize = 'index')
fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=['Non Repeated Booking', 'Repeated Booking'], y=repeated_cancelled[0]),
    go.Bar(name='Cancelled Booking', x=['Non Repeated Booking', 'Repeated Booking'], y=repeated_cancelled[1],marker_color = 'crimson')])

fig.update_layout(barmode='group', title = 'Percentage of Confirmed Booking and Cancelled Booking for Repeat & Non Repeat',
                 yaxis=dict(
                 title='Percentage of Booking (%)',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Booking Types',
                 titlefont_size=16,
                 tickfont_size=14,))

fig.show()

In [47]:
previously_cancelled =pd.crosstab(df['NumPrevCancellations'], df['BookingStatus'], normalize = 'index')
fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=previously_cancelled.index, y=previously_cancelled[0]),
    go.Bar(name='Cancelled Booking', x=previously_cancelled.index, y=previously_cancelled[1])])
fig.update_layout(barmode='group', title = 'Percentage of Previously Cancelled Booking',
                 yaxis=dict(
                 title='Percentage of Booking (%)',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Previously Cancelled',
                 titlefont_size=16,
                 tickfont_size=14,)) 
fig.show()

In [48]:
parking_cancel = pd.crosstab(df['Parking'], df['BookingStatus'], normalize = 'index')
parking_cancel

fig = go.Figure(data=[
    go.Bar(name='Confirmed Booking', x=parking_cancel.index, y=parking_cancel[0]),
    go.Bar(name='Cancelled Booking', x=parking_cancel.index, y=parking_cancel[1])])


fig.update_layout(barmode='group', title = 'Parking Space And Cancelled Booking',
                 yaxis=dict(
                 title='Percentage of Booking / Cancelled Booking (%)',
                 titlefont_size=16,
                 tickfont_size=14,)
                ,xaxis=dict(
                 title='Required Parking Space',
                 titlefont_size=16,
                 tickfont_size=14,)) 
fig.show()

In [1]:
!pip install -U tensorflow==2.10 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**The Determined training loop** will then invoke these functions automatically. These methods should be organized into a trial class, which is a user-defined Python class that inherits from **determined.keras.TFKerasTrial**

In [2]:
import keras
from determined.keras import TFKerasTrial, TFKerasTrialContext


class HotelBookingcancellation_exploratory(TFKerasTrial):
    def __init__(self, context: TFKerasTrialContext):
        # Initialize the trial class.
        pass

    def build_model(self):
        # Define and compile model graph.
        pass

    def build_training_data_loader(self):
        # Create the training data loader. This should return a keras.Sequence,
        # a tf.data.Dataset, or NumPy arrays.
        pass

    def build_validation_data_loader(self):
        # Create the validation data loader. This should return a keras.Sequence,
        # a tf.data.Dataset, or NumPy arrays.
        pass

The __init__ method is used to create our trial class, just like it is with any other Python class. TrialContext is the only parameter that Determined accepts. Information regarding the trial, such as the values of the hyperparameters to be used for training, is contained in the trial context. Although we don't currently need to access any of the trial context object's properties, we do so by assigning it to an instance variable so that we can utilise it later:

In [3]:
def __init__(self, context: TFKerasTrialContext):
    # Store trial context for later use.
    self.context = context

A compiled **tf.keras.Model** object is the result of the **build_model()** method. We may continue to use the Keras Sequential API in our version of build_model since it is used in the HotelBookingCancellation(my project) model code. The model must be wrapped by calling self.context.wrap_model() before it is compiled, and the optimizer must be wrapped by calling **self.context.wrap_optimizer(**). These are the only minimal differences.

In [4]:
def build_model(self):
    model = keras.Sequential(
        [
            keras.layers.Flatten(input_shape=(28, 28)),
            keras.layers.Dense(self.context.get_hparam("dense1"), activation="relu"),
            keras.layers.Dense(10),
        ]
    )

    # Wrap the model.
    model = self.context.wrap_model(model)

    # Create and wrap optimizer.
    optimizer = tf.keras.optimizers.Adam()
    optimizer = self.context.wrap_optimizer(optimizer)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    )
    return model

**Build_training_data_loader()** and **Build_validation_data_loader()** are the final two methods that need to be defined. These techniques are used by Determined to load the training dataset and the validation dataset, respectively.

Determined allows feeding data into a tf.keras model in three different ways: as a tf.keras.utils.A tf.data's sequence.Set of two NumPy arrays, or as a dataset.

In [5]:
def build_training_data_loader(self):
    train_images, train_labels = data.load_training_data()
    train_images = train_images / 255.0
    return train_images, train_labels

In [6]:
def build_validation_data_loader(self):
    test_images, test_labels = data.load_validation_data()
    test_images = test_images / 255.0
    return test_images, test_labels

**Train the model:-** We can use Determined to train a single instance of the model or to do a hyperparameter search.

In [7]:
train_model=["name:HotelBookingcancellation_exploratory",
              "global_batch_size: 32",
               "dense1: 128",
               "name: single",
               "metric: val_accuracy",
               "epochs: 5",
               "entrypoint: model_def:HotelBookingcancellation_exploratory"]

In [51]:
df.to_csv('DeterminedHotelBookingcancellation_training_testing_model.csv' ,index= False)